In [21]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn import preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import xgboost as xgb

from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/home/ubuntu/Envs/nlp/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
train = pd.read_csv('train.csv')
print(len(train))
train.head()

19579


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [253]:
test = pd.read_csv('test.csv')
print(len(test))
test.head()

8392


,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [254]:
label_enconder = preprocessing.LabelEncoder()
label_enconder.fit(train['author'])
train['label_encoded'] = label_enconder.transform(train['author'])
train.head()

,id,text,author,label_encoded,text_processed
0,id26305,"This process, however, afforded me no means of...",EAP,0,process howev afford mean ascertain dimens dun...
1,id17569,It never once occurred to me that the fumbling...,HPL,1,never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0,left hand gold snuff box which caper hill cut ...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2,love spring look windsor terrac sixteen fertil...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1,find noth els even gold superintend abandon at...


## Preprocessing function

In [255]:
def transformText(text):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    #filtered_words = [word for word in text.split() if word not in stops]
    filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Stemming
    text = gensim.parsing.preprocessing.stem_text(text)
    return text

## Creating preprocessing column

In [256]:
train['text_processed']=train['text'].apply(lambda x: transformText(x))
train.tail()

,id,text,author,label_encoded,text_processed
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,0,could have fanci while look it that some emin ...
19575,id08973,The lids clenched themselves together as if in...,EAP,0,the lid clench themselv togeth spasm
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,0,mai faut agir that sai frenchman never faint o...
19577,id17513,"For an item of news like this, it strikes us i...",EAP,0,for item new like thi strike wa veri coolli re...
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL,1,laid gnarl claw shoulder and seem that it shak...


In [257]:
test['text_processed']=test['text'].apply(lambda x: transformText(x))
print(len(test))
test.tail()

8392


,id,text,text_processed
8387,id11749,All this is now the fitter for my purpose.,all thi now the fitter for purpos
8388,id10526,I fixed myself on a wide solitude.,fix myself wide solitud
8389,id13477,It is easily understood that what might improv...,easili understood that what might improv close...
8390,id13761,"Be this as it may, I now began to feel the ins...",thi mai now began feel the inspir burn hope an...
8391,id04282,"Long winded, statistical, and drearily genealo...",long wind statist and drearili genealog some t...


## Train/Test split

In [258]:
x_train, x_test, y_train, y_test = train_test_split(train['text_processed'], train['label_encoded'], test_size = 0.2, random_state = 4)
true_label = np.array(y_test)
print("#" * 20 + " Some stats " + "#"*20)
print("Dataset training: {} uterances".format(x_train.shape[0]))
print("Dataset testing: {} uterances".format(x_test.shape[0]))
print("Different classes: {}".format(len(y_train.unique())))

#################### Some stats ####################
Dataset training: 15663 uterances
Dataset testing: 3916 uterances
Different classes: 3


## Generating features

In [259]:
# TFIDF features
tfv = TfidfVectorizer(ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
tfv.fit(list(x_train) + list(x_test))
x_train_tfv =  tfv.transform(x_train) 
x_test_tfv = tfv.transform(x_test)

In [260]:
# Count features
cnt = CountVectorizer(ngram_range=(1, 2))
cnt.fit(list(x_train) + list(x_test))
x_train_cnt =  cnt.transform(x_train) 
x_test_cnt = cnt.transform(x_test)

In [261]:
x_test_cnt.shape

(3916, 225243)

## Training XGBoost Classifer w/ early stopping on TFIDF features

In [262]:
clf = xgb.XGBClassifier(max_depth=4, n_estimators=1000, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
eval_set = [(x_test_tfv.tocsc(),y_test)]
clf.fit(x_train_tfv.tocsc(), y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
preds_proba = clf.predict_proba(x_test_tfv.tocsc())
preds = clf.predict(x_test_tfv.tocsc())

[0]	validation_0-mlogloss:1.08614
[1]	validation_0-mlogloss:1.07381
[2]	validation_0-mlogloss:1.06383
[3]	validation_0-mlogloss:1.05346
[4]	validation_0-mlogloss:1.04516
[5]	validation_0-mlogloss:1.03703
[6]	validation_0-mlogloss:1.02942
[7]	validation_0-mlogloss:1.0229
[8]	validation_0-mlogloss:1.0165
[9]	validation_0-mlogloss:1.01118
[10]	validation_0-mlogloss:1.00571
[11]	validation_0-mlogloss:1.00124
[12]	validation_0-mlogloss:0.995949
[13]	validation_0-mlogloss:0.991308
[14]	validation_0-mlogloss:0.98698
[15]	validation_0-mlogloss:0.983063
[16]	validation_0-mlogloss:0.978806
[17]	validation_0-mlogloss:0.975146
[18]	validation_0-mlogloss:0.971628
[19]	validation_0-mlogloss:0.968006
[20]	validation_0-mlogloss:0.964786
[21]	validation_0-mlogloss:0.961853
[22]	validation_0-mlogloss:0.958664
[23]	validation_0-mlogloss:0.955791
[24]	validation_0-mlogloss:0.953081
[25]	validation_0-mlogloss:0.949983
[26]	validation_0-mlogloss:0.947277
[27]	validation_0-mlogloss:0.944577
[28]	validation_0

[226]	validation_0-mlogloss:0.740931
[227]	validation_0-mlogloss:0.740395
[228]	validation_0-mlogloss:0.739921
[229]	validation_0-mlogloss:0.739499
[230]	validation_0-mlogloss:0.739053
[231]	validation_0-mlogloss:0.738529
[232]	validation_0-mlogloss:0.73812
[233]	validation_0-mlogloss:0.737461
[234]	validation_0-mlogloss:0.736981
[235]	validation_0-mlogloss:0.736513
[236]	validation_0-mlogloss:0.735929
[237]	validation_0-mlogloss:0.735467
[238]	validation_0-mlogloss:0.734974
[239]	validation_0-mlogloss:0.734398
[240]	validation_0-mlogloss:0.733794
[241]	validation_0-mlogloss:0.733342
[242]	validation_0-mlogloss:0.732799
[243]	validation_0-mlogloss:0.732224
[244]	validation_0-mlogloss:0.731697
[245]	validation_0-mlogloss:0.731177
[246]	validation_0-mlogloss:0.730604
[247]	validation_0-mlogloss:0.730102
[248]	validation_0-mlogloss:0.72962
[249]	validation_0-mlogloss:0.729142
[250]	validation_0-mlogloss:0.728581
[251]	validation_0-mlogloss:0.727999
[252]	validation_0-mlogloss:0.727525
[25

[448]	validation_0-mlogloss:0.663932
[449]	validation_0-mlogloss:0.66378
[450]	validation_0-mlogloss:0.663501
[451]	validation_0-mlogloss:0.663333
[452]	validation_0-mlogloss:0.662983
[453]	validation_0-mlogloss:0.662764
[454]	validation_0-mlogloss:0.66264
[455]	validation_0-mlogloss:0.66234
[456]	validation_0-mlogloss:0.662253
[457]	validation_0-mlogloss:0.66202
[458]	validation_0-mlogloss:0.661819
[459]	validation_0-mlogloss:0.661544
[460]	validation_0-mlogloss:0.661354
[461]	validation_0-mlogloss:0.66108
[462]	validation_0-mlogloss:0.660856
[463]	validation_0-mlogloss:0.660503
[464]	validation_0-mlogloss:0.660379
[465]	validation_0-mlogloss:0.660066
[466]	validation_0-mlogloss:0.659855
[467]	validation_0-mlogloss:0.65967
[468]	validation_0-mlogloss:0.659343
[469]	validation_0-mlogloss:0.659045
[470]	validation_0-mlogloss:0.658838
[471]	validation_0-mlogloss:0.658584
[472]	validation_0-mlogloss:0.658322
[473]	validation_0-mlogloss:0.65814
[474]	validation_0-mlogloss:0.657995
[475]	va

[671]	validation_0-mlogloss:0.621611
[672]	validation_0-mlogloss:0.621476
[673]	validation_0-mlogloss:0.62132
[674]	validation_0-mlogloss:0.621155
[675]	validation_0-mlogloss:0.621017
[676]	validation_0-mlogloss:0.620843
[677]	validation_0-mlogloss:0.620671
[678]	validation_0-mlogloss:0.620511
[679]	validation_0-mlogloss:0.620456
[680]	validation_0-mlogloss:0.620184
[681]	validation_0-mlogloss:0.620005
[682]	validation_0-mlogloss:0.619812
[683]	validation_0-mlogloss:0.619746
[684]	validation_0-mlogloss:0.61963
[685]	validation_0-mlogloss:0.619449
[686]	validation_0-mlogloss:0.619226
[687]	validation_0-mlogloss:0.619137
[688]	validation_0-mlogloss:0.61896
[689]	validation_0-mlogloss:0.618864
[690]	validation_0-mlogloss:0.618729
[691]	validation_0-mlogloss:0.618594
[692]	validation_0-mlogloss:0.618521
[693]	validation_0-mlogloss:0.61839
[694]	validation_0-mlogloss:0.618259
[695]	validation_0-mlogloss:0.618231
[696]	validation_0-mlogloss:0.618157
[697]	validation_0-mlogloss:0.617916
[698]

[894]	validation_0-mlogloss:0.593184
[895]	validation_0-mlogloss:0.593175
[896]	validation_0-mlogloss:0.593086
[897]	validation_0-mlogloss:0.59298
[898]	validation_0-mlogloss:0.592978
[899]	validation_0-mlogloss:0.592815
[900]	validation_0-mlogloss:0.59271
[901]	validation_0-mlogloss:0.592614
[902]	validation_0-mlogloss:0.592562
[903]	validation_0-mlogloss:0.592549
[904]	validation_0-mlogloss:0.592366
[905]	validation_0-mlogloss:0.59237
[906]	validation_0-mlogloss:0.592255
[907]	validation_0-mlogloss:0.592126
[908]	validation_0-mlogloss:0.592062
[909]	validation_0-mlogloss:0.591899
[910]	validation_0-mlogloss:0.591813
[911]	validation_0-mlogloss:0.591789
[912]	validation_0-mlogloss:0.591668
[913]	validation_0-mlogloss:0.591536
[914]	validation_0-mlogloss:0.591453
[915]	validation_0-mlogloss:0.591348
[916]	validation_0-mlogloss:0.591273
[917]	validation_0-mlogloss:0.591178
[918]	validation_0-mlogloss:0.591052
[919]	validation_0-mlogloss:0.590962
[920]	validation_0-mlogloss:0.590956
[921

In [251]:
print("Log-loss: {0:.3f}".format(log_loss(true_label, preds_proba)))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds,true_label)))

Log-loss: 0.584
Current Accuracy: 0.754


## Training XGBoost Classifer w/ early stopping on count features

In [185]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=3000, colsample_bytree=0.8, 
                        subsample=0.5, nthread=10, learning_rate=0.1)
eval_set = [(x_test_cnt.tocsc(),y_test)]
clf.fit(x_train_cnt.tocsc(), y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
#clf.fit(x_train_cnt.tocsc(), y_train)
preds_proba = clf.predict_proba(x_test_cnt.tocsc())
preds = clf.predict(x_test_cnt.tocsc())

[0]	validation_0-mlogloss:1.08762
[1]	validation_0-mlogloss:1.07743
[2]	validation_0-mlogloss:1.06763
[3]	validation_0-mlogloss:1.05862
[4]	validation_0-mlogloss:1.05107
[5]	validation_0-mlogloss:1.04399
[6]	validation_0-mlogloss:1.03705
[7]	validation_0-mlogloss:1.03069
[8]	validation_0-mlogloss:1.02466
[9]	validation_0-mlogloss:1.0192
[10]	validation_0-mlogloss:1.01402
[11]	validation_0-mlogloss:1.00887
[12]	validation_0-mlogloss:1.00422
[13]	validation_0-mlogloss:1.00024
[14]	validation_0-mlogloss:0.996283
[15]	validation_0-mlogloss:0.991872
[16]	validation_0-mlogloss:0.988181
[17]	validation_0-mlogloss:0.984671
[18]	validation_0-mlogloss:0.98093
[19]	validation_0-mlogloss:0.977677
[20]	validation_0-mlogloss:0.974193
[21]	validation_0-mlogloss:0.970882
[22]	validation_0-mlogloss:0.967865
[23]	validation_0-mlogloss:0.96494
[24]	validation_0-mlogloss:0.961749
[25]	validation_0-mlogloss:0.958871
[26]	validation_0-mlogloss:0.956115
[27]	validation_0-mlogloss:0.953397
[28]	validation_0-m

[226]	validation_0-mlogloss:0.735611
[227]	validation_0-mlogloss:0.735034
[228]	validation_0-mlogloss:0.734442
[229]	validation_0-mlogloss:0.733832
[230]	validation_0-mlogloss:0.733273
[231]	validation_0-mlogloss:0.732662
[232]	validation_0-mlogloss:0.732071
[233]	validation_0-mlogloss:0.73168
[234]	validation_0-mlogloss:0.731015
[235]	validation_0-mlogloss:0.730355
[236]	validation_0-mlogloss:0.729641
[237]	validation_0-mlogloss:0.729066
[238]	validation_0-mlogloss:0.728765
[239]	validation_0-mlogloss:0.728241
[240]	validation_0-mlogloss:0.727737
[241]	validation_0-mlogloss:0.72737
[242]	validation_0-mlogloss:0.726975
[243]	validation_0-mlogloss:0.726398
[244]	validation_0-mlogloss:0.725912
[245]	validation_0-mlogloss:0.72532
[246]	validation_0-mlogloss:0.724961
[247]	validation_0-mlogloss:0.724483
[248]	validation_0-mlogloss:0.724002
[249]	validation_0-mlogloss:0.723642
[250]	validation_0-mlogloss:0.722923
[251]	validation_0-mlogloss:0.72244
[252]	validation_0-mlogloss:0.721986
[253]

[449]	validation_0-mlogloss:0.650455
[450]	validation_0-mlogloss:0.65012
[451]	validation_0-mlogloss:0.649797
[452]	validation_0-mlogloss:0.649519
[453]	validation_0-mlogloss:0.649237
[454]	validation_0-mlogloss:0.64904
[455]	validation_0-mlogloss:0.648765
[456]	validation_0-mlogloss:0.648511
[457]	validation_0-mlogloss:0.648096
[458]	validation_0-mlogloss:0.647909
[459]	validation_0-mlogloss:0.647752
[460]	validation_0-mlogloss:0.647516
[461]	validation_0-mlogloss:0.647289
[462]	validation_0-mlogloss:0.647022
[463]	validation_0-mlogloss:0.646731
[464]	validation_0-mlogloss:0.646479
[465]	validation_0-mlogloss:0.646136
[466]	validation_0-mlogloss:0.645956
[467]	validation_0-mlogloss:0.645679
[468]	validation_0-mlogloss:0.645434
[469]	validation_0-mlogloss:0.645136
[470]	validation_0-mlogloss:0.644935
[471]	validation_0-mlogloss:0.644756
[472]	validation_0-mlogloss:0.644494
[473]	validation_0-mlogloss:0.644262
[474]	validation_0-mlogloss:0.643936
[475]	validation_0-mlogloss:0.643818
[47

[672]	validation_0-mlogloss:0.603852
[673]	validation_0-mlogloss:0.603821
[674]	validation_0-mlogloss:0.603681
[675]	validation_0-mlogloss:0.60334
[676]	validation_0-mlogloss:0.603248
[677]	validation_0-mlogloss:0.602955
[678]	validation_0-mlogloss:0.602779
[679]	validation_0-mlogloss:0.602637
[680]	validation_0-mlogloss:0.602454
[681]	validation_0-mlogloss:0.60236
[682]	validation_0-mlogloss:0.60216
[683]	validation_0-mlogloss:0.602048
[684]	validation_0-mlogloss:0.60189
[685]	validation_0-mlogloss:0.601779
[686]	validation_0-mlogloss:0.601475
[687]	validation_0-mlogloss:0.601307
[688]	validation_0-mlogloss:0.601185
[689]	validation_0-mlogloss:0.601115
[690]	validation_0-mlogloss:0.600978
[691]	validation_0-mlogloss:0.600877
[692]	validation_0-mlogloss:0.600693
[693]	validation_0-mlogloss:0.600482
[694]	validation_0-mlogloss:0.600316
[695]	validation_0-mlogloss:0.600063
[696]	validation_0-mlogloss:0.599876
[697]	validation_0-mlogloss:0.599716
[698]	validation_0-mlogloss:0.59953
[699]	

[895]	validation_0-mlogloss:0.574126
[896]	validation_0-mlogloss:0.574081
[897]	validation_0-mlogloss:0.573979
[898]	validation_0-mlogloss:0.573863
[899]	validation_0-mlogloss:0.573708
[900]	validation_0-mlogloss:0.573543
[901]	validation_0-mlogloss:0.573453
[902]	validation_0-mlogloss:0.573248
[903]	validation_0-mlogloss:0.573081
[904]	validation_0-mlogloss:0.572975
[905]	validation_0-mlogloss:0.572838
[906]	validation_0-mlogloss:0.572708
[907]	validation_0-mlogloss:0.572717
[908]	validation_0-mlogloss:0.572597
[909]	validation_0-mlogloss:0.572456
[910]	validation_0-mlogloss:0.572523
[911]	validation_0-mlogloss:0.572458
[912]	validation_0-mlogloss:0.572308
[913]	validation_0-mlogloss:0.572124
[914]	validation_0-mlogloss:0.572055
[915]	validation_0-mlogloss:0.571979
[916]	validation_0-mlogloss:0.571816
[917]	validation_0-mlogloss:0.571705
[918]	validation_0-mlogloss:0.571648
[919]	validation_0-mlogloss:0.571471
[920]	validation_0-mlogloss:0.571417
[921]	validation_0-mlogloss:0.5713
[92

...

[1983]	validation_0-mlogloss:0.523787
[1984]	validation_0-mlogloss:0.523838
[1985]	validation_0-mlogloss:0.523952
[1986]	validation_0-mlogloss:0.523965
[1987]	validation_0-mlogloss:0.523885
[1988]	validation_0-mlogloss:0.523799
[1989]	validation_0-mlogloss:0.52378
[1990]	validation_0-mlogloss:0.52381
[1991]	validation_0-mlogloss:0.523812
[1992]	validation_0-mlogloss:0.523802
[1993]	validation_0-mlogloss:0.523841
[1994]	validation_0-mlogloss:0.523819
[1995]	validation_0-mlogloss:0.52381
[1996]	validation_0-mlogloss:0.5238
[1997]	validation_0-mlogloss:0.523733
[1998]	validation_0-mlogloss:0.523755
[1999]	validation_0-mlogloss:0.5238
[2000]	validation_0-mlogloss:0.523862
[2001]	validation_0-mlogloss:0.523837
[2002]	validation_0-mlogloss:0.523839
[2003]	validation_0-mlogloss:0.523759
[2004]	validation_0-mlogloss:0.523711
[2005]	validation_0-mlogloss:0.523724
[2006]	validation_0-mlogloss:0.52378
[2007]	validation_0-mlogloss:0.523734
[2008]	validation_0-mlogloss:0.52368
[2009]	validation_0-m

[2200]	validation_0-mlogloss:0.522147
[2201]	validation_0-mlogloss:0.522154
[2202]	validation_0-mlogloss:0.522162
[2203]	validation_0-mlogloss:0.52217
[2204]	validation_0-mlogloss:0.522108
[2205]	validation_0-mlogloss:0.522093
[2206]	validation_0-mlogloss:0.52199
[2207]	validation_0-mlogloss:0.521976
[2208]	validation_0-mlogloss:0.522033
[2209]	validation_0-mlogloss:0.522014
[2210]	validation_0-mlogloss:0.52202
[2211]	validation_0-mlogloss:0.522149
[2212]	validation_0-mlogloss:0.522094
[2213]	validation_0-mlogloss:0.522029
[2214]	validation_0-mlogloss:0.522009
[2215]	validation_0-mlogloss:0.521912
[2216]	validation_0-mlogloss:0.521862
[2217]	validation_0-mlogloss:0.521771
[2218]	validation_0-mlogloss:0.521795
[2219]	validation_0-mlogloss:0.521732
[2220]	validation_0-mlogloss:0.521825
[2221]	validation_0-mlogloss:0.521826
[2222]	validation_0-mlogloss:0.5219
[2223]	validation_0-mlogloss:0.52188
[2224]	validation_0-mlogloss:0.521878
[2225]	validation_0-mlogloss:0.521874
[2226]	validation_

[2417]	validation_0-mlogloss:0.520513
[2418]	validation_0-mlogloss:0.520538
[2419]	validation_0-mlogloss:0.520569
[2420]	validation_0-mlogloss:0.520541
[2421]	validation_0-mlogloss:0.520559
[2422]	validation_0-mlogloss:0.520662
[2423]	validation_0-mlogloss:0.520601
[2424]	validation_0-mlogloss:0.520645
[2425]	validation_0-mlogloss:0.520642
[2426]	validation_0-mlogloss:0.520595
[2427]	validation_0-mlogloss:0.520619
[2428]	validation_0-mlogloss:0.520658
[2429]	validation_0-mlogloss:0.520624
[2430]	validation_0-mlogloss:0.520662
[2431]	validation_0-mlogloss:0.520596
[2432]	validation_0-mlogloss:0.520609
[2433]	validation_0-mlogloss:0.520579
[2434]	validation_0-mlogloss:0.520507
[2435]	validation_0-mlogloss:0.520577
[2436]	validation_0-mlogloss:0.520555
[2437]	validation_0-mlogloss:0.520564
[2438]	validation_0-mlogloss:0.520556
[2439]	validation_0-mlogloss:0.520544
[2440]	validation_0-mlogloss:0.520611
[2441]	validation_0-mlogloss:0.520567
[2442]	validation_0-mlogloss:0.520595
[2443]	valid

[2634]	validation_0-mlogloss:0.521902
[2635]	validation_0-mlogloss:0.52197
[2636]	validation_0-mlogloss:0.522007
[2637]	validation_0-mlogloss:0.521923
[2638]	validation_0-mlogloss:0.52202
[2639]	validation_0-mlogloss:0.521925
[2640]	validation_0-mlogloss:0.522009
[2641]	validation_0-mlogloss:0.521924
[2642]	validation_0-mlogloss:0.521845
[2643]	validation_0-mlogloss:0.521873
[2644]	validation_0-mlogloss:0.52188
[2645]	validation_0-mlogloss:0.521821
[2646]	validation_0-mlogloss:0.521847
[2647]	validation_0-mlogloss:0.521888
[2648]	validation_0-mlogloss:0.521855
[2649]	validation_0-mlogloss:0.52184
[2650]	validation_0-mlogloss:0.52184
[2651]	validation_0-mlogloss:0.521819
[2652]	validation_0-mlogloss:0.521855
[2653]	validation_0-mlogloss:0.521872
[2654]	validation_0-mlogloss:0.521818
[2655]	validation_0-mlogloss:0.521874
[2656]	validation_0-mlogloss:0.521848
[2657]	validation_0-mlogloss:0.521835
[2658]	validation_0-mlogloss:0.521912
[2659]	validation_0-mlogloss:0.521913
[2660]	validation

[2851]	validation_0-mlogloss:0.523323
[2852]	validation_0-mlogloss:0.523282
[2853]	validation_0-mlogloss:0.523221
[2854]	validation_0-mlogloss:0.523197
[2855]	validation_0-mlogloss:0.523179
[2856]	validation_0-mlogloss:0.523183
[2857]	validation_0-mlogloss:0.523178
[2858]	validation_0-mlogloss:0.52323
[2859]	validation_0-mlogloss:0.523268
[2860]	validation_0-mlogloss:0.523386
[2861]	validation_0-mlogloss:0.523326
[2862]	validation_0-mlogloss:0.523314
[2863]	validation_0-mlogloss:0.523422
[2864]	validation_0-mlogloss:0.52347
[2865]	validation_0-mlogloss:0.523478
[2866]	validation_0-mlogloss:0.523454
[2867]	validation_0-mlogloss:0.52351
[2868]	validation_0-mlogloss:0.523371
[2869]	validation_0-mlogloss:0.523363
[2870]	validation_0-mlogloss:0.523359
[2871]	validation_0-mlogloss:0.523362
[2872]	validation_0-mlogloss:0.523343
[2873]	validation_0-mlogloss:0.5233
[2874]	validation_0-mlogloss:0.523306
[2875]	validation_0-mlogloss:0.523335
[2876]	validation_0-mlogloss:0.523309
[2877]	validation

In [186]:
print("Log-loss: {0:.3f}".format(log_loss(true_label, preds_proba)))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds,true_label)))

Log-loss: 0.524
Current Accuracy: 0.791


## Generating submissions

In [238]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

,id,EAP,HPL,MWS


In [239]:
test['text_processed'][4247]

'stheticu ethix'

In [240]:
test['text'][4247]

'There was Æstheticus Ethix.'

In [242]:
query = test['text_processed'][4247]
query = cnt.transform([query])
preds_proba = clf.predict_proba(query.tocsc())
if not preds_proba.size:
    print("não tem reposta")
    preds_proba=np.array([[0.4, 0.3, 0.3]])
    print(preds_proba)
else:
    print("tem reposta")
    print(preds_proba)

não tem reposta
[[ 0.4  0.3  0.3]]


In [243]:
preds_proba

array([[ 0.4,  0.3,  0.3]])

In [244]:
print(len(test))

8392


In [245]:
my_sub

,id,EAP,HPL,MWS


In [246]:
for i in range(len(test)):
    query = test['text_processed'][i]
    query = cnt.transform([query])
    preds_proba = clf.predict_proba(query.tocsc())
    print(i)
    if not preds_proba.size:
        print("não tem reposta")
        preds_proba=np.array([[0.4, 0.3, 0.3]])
    my_sub.loc[i] = [test['id'][i], preds_proba[0][0], preds_proba[0][1], preds_proba[0][2]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [247]:
my_sub.tail()

,id,EAP,HPL,MWS
8387,id11749,0.623781,0.093822,0.282396
8388,id10526,0.075611,0.065202,0.859187
8389,id13477,0.762924,0.089913,0.147163
8390,id13761,0.083209,0.021354,0.895437
8391,id04282,0.105051,0.892457,0.002492


In [248]:
my_sub.to_csv('roberto_new.csv',index=False)